<a href="https://colab.research.google.com/github/dlvvkfl88/MSAISCHOOL/blob/main/MS_AI_SCHOOL_64%EC%9D%BC%EC%B0%A8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 딥러닝 학습법  부적합 , 과적합

## 드롭아웃 - 노드 갯수를 무작위로 어느정도 없애 학습 시간을 빠르게 함

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# 드롭아웃 적용한 신경망 모델 정의

In [22]:
class DropoutNet(nn.Module) :
    def __init__(self) :
        super(DropoutNet, self).__init__()
        ### 사용할 데이터 크기가 28x28 -> 784 1차원
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5) # p=0.5 50%
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x) :
        x = x.view(x.size(0), -1) # 입력 데이터 크기 조절 -> 1차원 으로 펼침
        x = torch.relu(self.fc1(x)) # self.fc1(x) -> relu 적용
        x = self.dropout(x)
        x = self.fc2(x)

        return x

## 드롭아웃 적용x 신경망 모델 정의

In [23]:
class NonDropoutNet(nn.Module) :
    def __init__(self) :
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x) :
        x = x.view(x.size(0), -1) # 입력 데이터 크기 조절 -> 1차원 으로 펼침
        x = torch.relu(self.fc1(x)) # self.fc1(x) -> relu 적용
        x = self.fc2(x)

        return x

## MNIST 데이터셋 로드 및 전처리

In [24]:
train_transform = transforms.Compose([
#     transforms.AutoAugment(),
    transforms.ToTensor(),
    ### transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    transforms.Normalize((0.5,),(0.3,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.3,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 드롭아웃 적용모델 초기화, loss 옵티마이저 정의

In [26]:
# 드롭아웃 적용 모델 초기화
dropout_model = DropoutNet()
print(dropout_model)

# loss 함수, 옵티마이저 설정
dropout_criterion = nn.CrossEntropyLoss()
dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.001)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


# 드롭아웃 적용모델 학습, 평가 정의

In [27]:
# 드롭아웃 적용 모델 학습 코드
for epoch in range(10) :
    dropout_model.train() #### -> train 모드 선언
    for images, labels in train_loader :
        dropout_optimizer.zero_grad()
        dropout_output = dropout_model(images)
        dropout_loss = dropout_criterion(dropout_output, labels)
        dropout_loss.backward()
        dropout_optimizer.step()

# 드롭아웃 적용 모델 평가 코드
dropout_model.eval()
with torch.no_grad() :
    dropout_correct = 0
    dropout_total = 0

    for images, labels in test_loader :
        test_out = dropout_model(images)
        _, dropout_pre = torch.max(test_out.data, 1)
        dropout_total += images.size(0)
        dropout_correct += (dropout_pre == labels).sum().item()

    print("드롭아웃 적용 모델 정확도 >> {:.2f}%".format(100 * dropout_correct / dropout_total))

드롭아웃 적용 모델 정확도 >> 91.07%


## 드롭아웃 비적용

In [28]:
# 모델 호출
non_dropout_model = NonDropoutNet()

# 옵티마이저 loss function 정의
non_dropout_criterion = nn.CrossEntropyLoss()
non_dropout_optimizer = optim.SGD(non_dropout_model.parameters(), lr=0.001)

# 드롭아웃 적용되지 않은 모델 학습
# 시작 시간 구하시고
for epoch in range(10) :
    non_dropout_model.train()
    for images, labels in train_loader :
        non_dropout_optimizer.zero_grad()
        no_output = non_dropout_model(images)
        no_losss = non_dropout_criterion(no_output, labels)
        no_losss.backward()
        non_dropout_optimizer.step()

# 현재시간 - 시작 시간
# 드롭아웃 적용하지 않은 모델 평가
non_dropout_model.eval()
with torch.no_grad()  :
    no_correct = 0
    no_total = 0

    for images, labels in test_loader :
        ouput = non_dropout_model(images)
        _, pred = torch.max(ouput.data, 1)
        no_total += labels.size(0)
        no_correct += (pred == labels).sum().item()

    print("드롭아웃 적용하지 않은 모델 정확도 >> {:.2f}%".format(100 * no_correct / no_total))

드롭아웃 적용하지 않은 모델 정확도 >> 91.11%


## 라벨 스무딩 - 오버피팅 해결방법 중 하나


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from torchvision.transforms import ToTensor

### 사용자 정의 손실함수 (라벨 스무딩 적용 크로스 엔트로피)

In [31]:
class LabelSmothingLoss(nn.Module) : ### 사용자 정의 손실 함수
    def __init__(self, num_classes, smothing=0.0) :
        super(LabelSmothingLoss, self).__init__()
        self.num_classes = num_classes  #### -> 분류 문제 : 클래스 개수
        self.smothing = smothing        #### -> 라벨 적용되는 스무딩 정도 제어 값 -> 기본 0.0 스무딩 적용 x
        self.confidence = 1.0 - smothing  #### -> smothing 보정값 (정답에 대한 신뢰도 점수)

    def forward(self, pred, target) : # pred : 예측값 , target : 정답지
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1),1) # 원-핫 인코딩 텐서 생성
        smoth_label = one_hot * self.confidence + (1 - one_hot) * self.smothing / (self.num_classes - 1)
        #### 원-핫 인코딩 텐서에서 라벨 스무딩을 적용 -> 신뢰도 점수
        loss = torch.sum(-smoth_label * torch.log_softmax(pred, dim=1), dim=1)
        #### 스무딩 처리된 라벨과 예측 확률간 교차 엔트로피 사용해서 loss 계산 -> log_softmax 확률 과 라벨스무딩 요소 곱을 통해 수행

        return torch.mean(loss) # 평균 손실 구해 주고 반환

# Mix up training
- Data Augmentation 기법의 일종이다
두 데이터 라벨을 일정비율로 섞어 새로운 데이터 생성